In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
from pyspark.sql.types import StructType, StringType, DoubleType, IntegerType

In [2]:
spark = SparkSession.builder \
    .appName("FraudDetection") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/15 18:01:27 WARN Utils: Your hostname, NolanPC, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/08/15 18:01:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/15 18:01:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
schema = StructType() \
    .add("transaction_id", StringType()) \
    .add("user_id", StringType()) \
    .add("merchant", StringType()) \
    .add("amount", DoubleType()) \
    .add("timestamp", StringType()) \
    .add("is_fraud", IntegerType())

stream_df = spark.readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

parsed_df = stream_df.select(sf.from_json(stream_df.value, schema).alias("data")) \
                        .select("data.*")

parsed_df = parsed_df.withColumn("timestamp", sf.to_timestamp("timestamp"))

TARGET_COUNT = 3_000_000
accumulated_count = {"count": 0}

def process_batch(df, epoch_id):
    batch_count = df.count()
    accumulated_count["count"] += batch_count
    print(f"Processed batch: {batch_count} rows, Total: {accumulated_count["count"]} rows")

    df.write.mode("append").parquet("/home/nolan/fraud/data/transactions")

    if accumulated_count["count"] >= TARGET_COUNT:
        print("Target reached - stopping stream")
        for q in spark.streams.active:
            q.stop()

query = parsed_df.writeStream \
    .foreachBatch(process_batch) \
    .outputMode("append") \
    .option("checkpointLocation", "/home/nolan/fraud/data/checkpoints") \
    .trigger(processingTime="2 seconds") \
    .start()

query.awaitTermination()

25/08/15 18:14:11 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
25/08/15 18:14:11 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Processed batch: 0 rows, Total: 0 rows
Processed batch: 500 rows, Total: 500 rows
Processed batch: 1000 rows, Total: 1500 rows
Processed batch: 1000 rows, Total: 2500 rows
Processed batch: 1000 rows, Total: 3500 rows
Processed batch: 1000 rows, Total: 4500 rows
Processed batch: 1500 rows, Total: 6000 rows
Processed batch: 1000 rows, Total: 7000 rows
Processed batch: 1000 rows, Total: 8000 rows
Processed batch: 1000 rows, Total: 9000 rows


25/08/15 18:14:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


Processed batch: 1000 rows, Total: 10000 rows
Processed batch: 1000 rows, Total: 11000 rows
Processed batch: 1000 rows, Total: 12000 rows


25/08/15 18:14:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


...

Processed batch: 1000 rows, Total: 2992500 rows
Processed batch: 1000 rows, Total: 2993500 rows
Processed batch: 1000 rows, Total: 2994500 rows
Processed batch: 1000 rows, Total: 2995500 rows
Processed batch: 500 rows, Total: 2996000 rows
Processed batch: 1000 rows, Total: 2997000 rows
Processed batch: 1000 rows, Total: 2998000 rows
Processed batch: 1000 rows, Total: 2999000 rows


25/08/15 19:52:08 WARN DAGScheduler: Failed to cancel job group 4feb2fb9-a4a5-4292-89c7-dc885c0a9893. Cannot find active jobs for it.
ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/home/nolan/anaconda3/envs/fraud/lib/python3.12/site-packages/py4j/clientserver.py", line 644, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nolan/anaconda3/envs/fraud/lib/python3.12/site-packages/pyspark/sql/utils.py", line 165, in call
    raise e
  File "/home/nolan/anaconda3/envs/fraud/lib/python3.12/site-packages/pyspark/sql/utils.py", line 162, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "/tmp/ipykernel_971/2562846172.py", line 33, in process_batch
    q.stop()
  File "/home/nolan/anaconda3/envs/fraud/lib/python3.12/site-packages/pyspark/sql/streaming/query.py", line 37

Processed batch: 1000 rows, Total: 3000000 rows
Target reached - stopping stream


Exception in thread "stream execution thread for [id = 8027a5d4-d48d-4e54-a67c-15a0444c8d95, runId = 4feb2fb9-a4a5-4292-89c7-dc885c0a9893]" java.lang.StackOverflowError
	at java.base/java.util.regex.Pattern$Loop.match(Pattern.java:4894)
	at java.base/java.util.regex.Pattern$GroupTail.match(Pattern.java:4820)
	at java.base/java.util.regex.Pattern$BranchConn.match(Pattern.java:4698)
	at java.base/java.util.regex.Pattern$CharProperty.match(Pattern.java:3931)
	at java.base/java.util.regex.Pattern$Branch.match(Pattern.java:4734)
	at java.base/java.util.regex.Pattern$GroupHead.match(Pattern.java:4789)
	at java.base/java.util.regex.Pattern$Loop.match(Pattern.java:4898)
	at java.base/java.util.regex.Pattern$GroupTail.match(Pattern.java:4820)
	at java.base/java.util.regex.Pattern$BranchConn.match(Pattern.java:4698)
	at java.base/java.util.regex.Pattern$CharProperty.match(Pattern.java:3931)
	at java.base/java.util.regex.Pattern$Branch.match(Pattern.java:4734)
	at java.base/java.util.regex.Patter

rn.java:4734)
	at java.base/java.util.regex.Pattern$GroupHead.match(Pattern.java:4789)
	at java.base/java.util.regex.Pattern$Loop.match(Pattern.java:4898)
	at java.base/java.util.regex.Pattern$GroupTail.match(Pattern.java:4820)
	at java.base/java.util.regex.Pattern$BranchConn.match(Pattern.java:4698)
	at java.base/java.util.regex.Pattern$CharProperty.match(Pattern.java:3931)
	at java.base/java.util.regex.Pattern$Branch.match(Pattern.java:4734)
	at java.base/java.util.regex.Pattern$GroupHead.match(Pattern.java:4789)
	at java.base/java.util.regex.Pattern$Loop.match(Pattern.java:4898)
	at java.base/java.util.regex.Pattern$GroupTail.match(Pattern.java:4820)
	at java.base/java.util.regex.Pattern$BranchConn.match(Pattern.java:4698)
	at java.base/java.util.regex.Pattern$CharProperty.match(Pattern.java:3931)
	at java.base/java.util.regex.Pattern$Branch.match(Pattern.java:4734)
	at java.base/java.util.regex.Pattern$GroupHead.match(Pattern.java:4789)
	at java.base/java.util.regex.Pattern$Loop.ma

In [5]:
for q in spark.streams.active:
    print(q.status)

In [6]:
df = spark.read.parquet("/home/nolan/fraud/data/transactions")
print(f"Total rows: {df.count()}")
df.show(5)

Total rows: 3000000
+--------------------+---------+-------------+------+--------------------+--------+
|      transaction_id|  user_id|     merchant|amount|           timestamp|is_fraud|
+--------------------+---------+-------------+------+--------------------+--------+
|74256537-0937-434...|user_5703|     clothing| 12.97|2025-08-15 19:22:...|       0|
|ac3815a5-dd38-430...| user_501|  electronics|726.74|2025-08-15 19:22:...|       0|
|d7341b09-75f6-40a...|user_4684|subscriptions|  25.4|2025-08-15 19:22:...|       0|
|b1d6f587-18ae-49b...|user_6629|     clothing|253.08|2025-08-15 19:22:...|       0|
|95e01b16-8562-4e6...|user_1580|  electronics|212.71|2025-08-15 19:22:...|       0|
+--------------------+---------+-------------+------+--------------------+--------+
only showing top 5 rows


In [7]:
spark.stop()